## Calculation of Cu211 with CONQUEST code

In [ ]:
import os
from ase.io import read

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 10 /opt/chemistry/conquest/dev/bin/Conquest'
os.environ['CQ_PP_PATH'] = '/opt/chemistry/conquest/dev/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = 'mpirun -np 1 /opt/chemistry/conquest/dev/bin/MakeIonFiles'

In [ ]:
from ase.build import fcc211, add_vacuum
a=3.616 # angstrom, from medium cutoff100 k888 condition for bulk calculation
struct = fcc211('Cu', (3,3,3), a=a, vacuum=1e-20, orthogonal=True)
add_vacuum(struct, vacuum=20)

In [ ]:
from my_visualize import print_xyz
print('Number of atoms = {} '.format(len(struct.positions)))
print_xyz(struct)

#### Directory for storing calculation files

In [ ]:
def mk_workdir(workingdir):
    
    # Test if `working_directory` exists ? If not create it
    if ( not os.path.isdir(workingdir) ):
        os.makedirs(workingdir)
    return

In [ ]:
def set_Basis(basis_size, xc):
    basis = {
             'Cu': {'gen_basis'            : True,
                    'basis_size'           : basis_size,
                    "pseudopotential_type": 'hamann',
                    'xc'                   : xc,
                    },
             'O' : { 'gen_basis'            : True,
                     'basis_size'           : basis_size,
                     'pseudopotential_type' : 'hamann',
                     'xc'                   : xc },             
             'H' : { 'gen_basis'            : True,
                     'basis_size'           : basis_size,
                     'pseudopotential_type' : 'hamann',
                     'xc'                   : xc },
             'C' : { 'gen_basis'            : True,
                     'basis_size'           : basis_size,
                     'pseudopotential_type' : 'hamann',
                     'xc'                   : xc },         
              }
    return(basis)

In [ ]:
def set_Calculator_CQ(struct, basis_size, fxc, cutoff, kpoints):
    from ase.calculators.conquest import Conquest
    
    basis = set_Basis(basis_size,fxc)
    
    conquest_flags = {'IO.WriteOutToASEFile': True}

    # Flags for atomic positions optimisation
    conquest_flags.update({'AtomMove.TypeOfRun'   : 'sqnm',  # optimization algorithm
                           'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                           'AtomMove.ReuseDM'     :  True,    
                           'AtomMove.AppendCoords':  True,
                           })    

    calc = Conquest(directory      = working_directory,
                    grid_cutoff    = cutoff,
                    scf_tolerance=1.0e-7,  # Ha
                    self_consistent= True,
                    xc    = fxc,
                    basis = basis,
                    kpts  = kpoints,
                    nspin = 1,
                    **conquest_flags)

    struct.calc = calc
    return(struct)

In [ ]:
# Create workdir
working_directory = 'Cu211_opt'
mk_workdir(working_directory)

In [ ]:
struct = set_Calculator_CQ(struct,
                           'medium',
                           'PBE',
                           cutoff = 100.0,
                           kpoints = [4,3,1]
                          )

In [ ]:
import time
t1 = time.time()

# Create input files
struct.calc.write_input(struct)

t2 = time.time()
time_elapsed = t2 - t1
print('Total_time_elapsed = {:.3f} sec'.format(time_elapsed))